# Assignment 4
* Unit: SIT720
* Name: Bryon Baker
* Student #: 85031775

In [1]:
#!pip3 install pandas
#!pip3 install numpy

In [2]:
import sys
import warnings
import random
import pandas as pd # dataframe manipulation
import numpy as np # linear algebra

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

In [3]:
# Seed the random nunmber generator to ensure K-Means randomisation works properly.
random.seed(0)

# Constant used for train_test_split so there is predicatability across runs
global_random_state=11

print(f"Seeding random number generator. Random number is: {random.random()}")

Seeding random number generator. Random number is: 0.8444218515250481


# Helper Functions
This section defines any helper functions used in the code below

In [4]:
# Calculate the inverse of log(x+1) and assign any negative value to zero
def inverse_log_transform( vals ):
    vals_x = np.exp(vals)-1
    vals_x[vals_x < 0] = 0
    
    return vals_x

In [5]:
# A helper function to standardise all the values in the dataset with a mean=0 and SD=1
def standardise_dataset( dataset_name, dataset, column_headings ):
    # Standardise all attributes to zero mean and SD 1
    scaler = StandardScaler(copy=False, with_mean=True, with_std=True)
    scaler.fit(dataset)
    result = scaler.transform(dataset)
    # print(f"Transformed Data Frame:\n{SVM_STFWI_df}")
    m = np.mean(result, axis=0)
    s = np.std(result, axis=0)
    #print("Checking transformation")
    #print(f"Column  means:\n{m}")
    #print(f"Column  SD:\n{s}")

    # The StandardScaler converts to an array. Convert back to a DataFrame
    result_df = pd.DataFrame(result)
    result_df.columns = column_headings     # Reassign the column names.
    # print(result_df.head())
    
    return result_df

In [6]:
# Helper to calculate the RMSE
def calc_rmse( y, y_hat ):
    # Perform the inverse transform of y and h_hat
    yy = np.exp(y)-1
    yy[yy < 0] = 0
    
    yy_hat = np.exp(y_hat)-1
    yy_hat[yy_hat < 0] = 0
        
    rmse = mean_squared_error(yy, yy_hat, squared=False)
    
    return rmse

In [7]:
from pathlib import Path 


def save_dataset( filename, dataset ):
    return   # This is bypassed for the final assignment submission in case this causes the notbook to crash on a different machine
    file_dir = "~/datasets/SIT720/Ass4/transformed/"
    filepath = Path(file_dir)  
    filepath.parent.mkdir(parents=True, exist_ok=True)
    
    full_filename = file_dir + filename + ".csv"
    dataset.to_csv(full_filename)

# Load the source dataset
Dataset is stored in public git repo. COnfigure the ssl context in order to access github raw data files.

In [8]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://raw.githubusercontent.com/bryonbaker/datasets/main/SIT720/Ass4/forestfires.csv'
full_df = pd.read_csv(url)
print(f"{full_df.head()}\n")

   X  Y month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain  area
0  7  5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0   0.0
1  7  4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0   0.0
2  7  4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0   0.0
3  8  6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2   0.0
4  8  6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0   0.0



# Question 1

Read the article and reproduce the RMSE results presented in Table 3 using Python modules and packages (including your own script or customised codes). Write a report summarising the dataset, used ML methods, experiment protocol and results including variations, if any. During reproducing the results:               (15 Marks)

1. you should use the same set of features used by the authors.
2. you should use the same classifier with exact parameter values.
3. you should use the same training/test splitting approach as used by the authors.
4. you should use the same pre/post processing, if any, used by the authors.
5. you should report the same performance metric (RMSE) as shown in Table 3.

# Pre Process the Data
1. One hot encode the day and month attributes.
2. Ordinal encode the day/month for DT, MR, RF
3. Transform the area to be y = ln(area+1)

## Preprocess area
Reduce the skewness of area as per procedure on page 5.
This will be used to train the model, but it is important to take the inverse log after prediction so as to find the predicted area. 


In [9]:
# Check the dataset has no NaNs.
if full_df.isnull().sum().sum() == 0:
    print("Data does not have any missing values.")
else:
    print("Data has missing values and needs pre processing.")
    print("Stopping....")
    quit() 

Data does not have any missing values.


In [10]:
area_df = pd.DataFrame(full_df["area"])
area_df["xfrmd_area"] = full_df["area"] +1                 # Create a new area column that will contain the ln(area+1)
print(area_df.head())
area_df["xfrmd_area"] = np.log(area_df['xfrmd_area'])      # Take the ln(area+1)
print(area_df.head()) 

   area  xfrmd_area
0   0.0         1.0
1   0.0         1.0
2   0.0         1.0
3   0.0         1.0
4   0.0         1.0
   area  xfrmd_area
0   0.0         0.0
1   0.0         0.0
2   0.0         0.0
3   0.0         0.0
4   0.0         0.0


### 1-of-C Encoding

In [11]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', drop=None, dtype='int')

In [12]:
# One-hot encode month
transformed = ohe.fit_transform(full_df[['month']])
ohe_month_df = pd.DataFrame(transformed.toarray())
ohe_month_df.columns = ohe.categories_[0]

In [13]:
# One hot encode day
transformed = ohe.fit_transform(full_df[['day']])
ohe_day_df = pd.DataFrame(transformed.toarray())
ohe_day_df.columns = ohe.categories_[0]

In [14]:
# encoded_df holds all the training.test data that has been 1-of-C encoded.
# unencoded
ohe_encoded_df = full_df.drop(['month','day','area'], axis=1)
ohe_encoded_df = pd.concat([ohe_encoded_df, ohe_month_df], axis=1)
ohe_encoded_df = pd.concat([ohe_encoded_df, ohe_day_df], axis=1)
ohe_encoded_df.head(20)

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,nov,oct,sep,fri,mon,sat,sun,thu,tue,wed
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,1,0,0,0,0,0,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,1,0,0,0,0,0,0,1,0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,1,0,0,0,1,0,0,0,0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,1,0,0,0,0,0,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,0,0,1,0,0,0
5,8,6,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,...,0,0,0,0,0,0,1,0,0,0
6,8,6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,...,0,0,0,0,1,0,0,0,0,0
7,8,6,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,...,0,0,0,0,1,0,0,0,0,0
8,8,6,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,...,0,0,1,0,0,0,0,0,1,0
9,7,5,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,...,0,0,1,0,0,1,0,0,0,0


### Ordinal Encoding

In [15]:
# Ordinal encode month
from sklearn.preprocessing import OrdinalEncoder

ore = OrdinalEncoder(dtype='int')

transformed = ore.fit_transform(full_df[['month']])
ord_month_df = pd.DataFrame(transformed)
ord_month_df.columns = ['month']

In [16]:
# Ordinal encode month
from sklearn.preprocessing import OrdinalEncoder

ore = OrdinalEncoder(dtype='int')

transformed = ore.fit_transform(full_df[['day']])
ord_day_df = pd.DataFrame(transformed)
ord_day_df.columns = ['day']

In [17]:
ord_encoded_df = full_df     # RF and DT need the unencoded dataframe
ord_encoded_df = full_df.drop(['month','day','area'], axis=1)
ord_encoded_df = pd.concat([ord_encoded_df, ord_month_df], axis=1)
ord_encoded_df = pd.concat([ord_encoded_df, ord_day_df], axis=1)
ord_encoded_df.head(20)

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,month,day
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,7,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,10,5
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,10,2
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,7,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,7,3
5,8,6,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,1,3
6,8,6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,1,1
7,8,6,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,1,1
8,8,6,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,11,5
9,7,5,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,11,2


In [18]:
del full_df         # Make sure we don't accidentally use full_df later

## Create Sub-Datasets

Now we have all the raw ingredients for all datasets, create the sub datasets that the different models will need.

In [19]:
# Define the sub datasets from the transformed dataset

# Get the 1-of-C encoded column names for month and day
month_cols = ohe_month_df.columns.tolist()
day_cols = ohe_day_df.columns.tolist()

# Construct the dataframe column names
STFWI_encoded_cols = ['X','Y','FFMC','DMC','DC','ISI']
STFWI_encoded_cols = STFWI_encoded_cols + month_cols + day_cols
STM_encoded_cols = ['X','Y','temp','RH','wind','rain']
STM_encoded_cols = STM_encoded_cols + month_cols + day_cols
FWI_encoded_cols = ['FFMC','DMC','DC','ISI']
M_encoded_cols = ['temp','RH','wind','rain']

# Construct the dataframe column names
STFWI_unencoded_cols = ['X','Y','month','day','FFMC','DMC','DC','ISI']
STM_unencoded_cols = ['X','Y','month','day','temp','RH','wind','rain']
FWI_unencoded_cols = ['FFMC','DMC','DC','ISI']
M_unencoded_cols = ['temp','RH','wind','rain']


In [20]:
# create feature tables for MR, SVM, and NN
STFWI_ohe_encoded_df = ohe_encoded_df[STFWI_encoded_cols]
STM_ohe_encoded_df = ohe_encoded_df[STM_encoded_cols]
FWI_ohe_encoded_df = ohe_encoded_df[FWI_encoded_cols]
M_ohe_encoded_df = ohe_encoded_df[M_encoded_cols]
# Create the feature tables for RF and DT
STFWI_ord_encoded_df = ord_encoded_df[STFWI_unencoded_cols]
STM_ord_encoded_df = ord_encoded_df[STM_unencoded_cols]
FWI_ord_encoded_df = ord_encoded_df[FWI_unencoded_cols]
M_ord_encoded_df = ord_encoded_df[M_unencoded_cols]
# Create the transformed area ln(area+1)
y_xformed = area_df["xfrmd_area"]
y_vanilla = area_df["area"]

In [21]:
print("SVM, MR, and NN Dataset columns:")
print("="*16)
print(f"STFWI Dataset columns:\n{STFWI_ohe_encoded_df.columns}\n")
print(f"STM Dataset columns:\n{STM_ohe_encoded_df.columns}\n")
print(f"FWI Dataset columns:\n{FWI_ohe_encoded_df.columns}\n")
print(f"M Dataset columns:\n{M_ohe_encoded_df.columns}\n")
print("="*16)
print("DT and RF Dataset columns:")
print("="*16)
print(f"STFWI Dataset columns:\n{STFWI_ord_encoded_df.columns}\n")
print(f"STM Dataset columns:\n{STM_ord_encoded_df.columns}\n")
print(f"FWI Dataset columns:\n{FWI_ord_encoded_df.columns}\n")
print(f"M Dataset columns:\n{M_ord_encoded_df.columns}\n")

SVM, MR, and NN Dataset columns:
STFWI Dataset columns:
Index(['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'apr', 'aug', 'dec', 'feb', 'jan',
       'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep', 'fri', 'mon', 'sat',
       'sun', 'thu', 'tue', 'wed'],
      dtype='object')

STM Dataset columns:
Index(['X', 'Y', 'temp', 'RH', 'wind', 'rain', 'apr', 'aug', 'dec', 'feb',
       'jan', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep', 'fri', 'mon',
       'sat', 'sun', 'thu', 'tue', 'wed'],
      dtype='object')

FWI Dataset columns:
Index(['FFMC', 'DMC', 'DC', 'ISI'], dtype='object')

M Dataset columns:
Index(['temp', 'RH', 'wind', 'rain'], dtype='object')

DT and RF Dataset columns:
STFWI Dataset columns:
Index(['X', 'Y', 'month', 'day', 'FFMC', 'DMC', 'DC', 'ISI'], dtype='object')

STM Dataset columns:
Index(['X', 'Y', 'month', 'day', 'temp', 'RH', 'wind', 'rain'], dtype='object')

FWI Dataset columns:
Index(['FFMC', 'DMC', 'DC', 'ISI'], dtype='object')

M Dataset columns:
Index(['temp', 'R

## Decision Tree Dataset Preparation

In [22]:
DT_STFWI_df = STFWI_ord_encoded_df
DT_STM_df = STM_ord_encoded_df
DT_FWI_df = FWI_ord_encoded_df
DT_M_df = M_ord_encoded_df

save_dataset( "DT_STFWI_df", DT_STFWI_df )
save_dataset( "DT_STM_df", DT_STM_df )
save_dataset( "DT_FWI_df", DT_FWI_df )
save_dataset( "DT_M_df", DT_M_df )



## Random Forrest Dataset Preparation

In [23]:
RF_STFWI_df = STFWI_ord_encoded_df
RF_STM_df = STM_ord_encoded_df
RF_FWI_df = FWI_ord_encoded_df
RF_M_df = M_ord_encoded_df

# Save the dataset to a file for manual exploration and verifiaction of feature engineering.
save_dataset( "RF_STFWI_df", RF_STFWI_df )
save_dataset( "RF_STM_df", RF_STM_df )
save_dataset( "RF_FWI_df", RF_FWI_df )
save_dataset( "RF_M_df", RF_M_df )

## SVM Data Set Preparation

In [24]:
# Standardise all attributes to zero mean and SD 1
SVM_STFWI_df = STFWI_ohe_encoded_df
SVM_STM_df = STM_ohe_encoded_df
SVM_FWI_df = FWI_ohe_encoded_df
SVM_M_df = M_ohe_encoded_df

SVM_STFWI_df = standardise_dataset("SVM_STFWI_df", SVM_STFWI_df, STFWI_encoded_cols)
SVM_STM_df = standardise_dataset("SVM_STM_df", SVM_STM_df, STM_encoded_cols)
SVM_FWI_df = standardise_dataset("SVM_FWI_df", SVM_FWI_df, FWI_encoded_cols)
SVM_M_df = standardise_dataset("SVM_M_df", SVM_M_df, M_encoded_cols)

# Save the dataset to a file for manual exploration and verifiaction of feature engineering.
save_dataset( "SVM_STFWI_df", SVM_STFWI_df )
save_dataset( "SVM_STM_df", SVM_STM_df )
save_dataset( "SVM_FWI_df", SVM_FWI_df )
save_dataset( "SVM_M_df", SVM_M_df )

## Multiple Rgression Data Set Preparation

In [25]:
MR_STFWI_df = STFWI_ohe_encoded_df
MR_STM_df = STM_ohe_encoded_df
MR_FWI_df = FWI_ohe_encoded_df
MR_M_df = M_ohe_encoded_df

MR_STFWI_df = standardise_dataset("MR_STFWI_df", MR_STFWI_df, STFWI_encoded_cols)
MR_STM_df = standardise_dataset("MR_STM_df", MR_STM_df, STM_encoded_cols)
MR_FWI_df = standardise_dataset("MR_FWI_df", MR_FWI_df, FWI_encoded_cols)
MR_M_df = standardise_dataset("MR_M_df", MR_M_df, M_encoded_cols)

# Save the dataset to a file for manual exploration and verifiaction of feature engineering.
save_dataset( "MR_STFWI_df", MR_STFWI_df )
save_dataset( "MR_STM_df", MR_STM_df )
save_dataset( "MR_FWI_df", MR_FWI_df )
save_dataset( "MR_M_df", MR_M_df )

## Neural Network Dataset Preparation



In [26]:
NN_STFWI_df = STFWI_ohe_encoded_df
NN_STM_df = STM_ohe_encoded_df
NN_FWI_df = FWI_ohe_encoded_df
NN_M_df = M_ohe_encoded_df

NN_STFWI_df = standardise_dataset("NN_STFWI_df", NN_STFWI_df, STFWI_encoded_cols)
NN_STM_df = standardise_dataset("NN_STM_df", NN_STM_df, STM_encoded_cols)
NN_FWI_df = standardise_dataset("NN_FWI_df", NN_FWI_df, FWI_encoded_cols)
NN_M_df = standardise_dataset("NN_M_df", NN_M_df, M_encoded_cols)

# Save the dataset to a file for manual exploration and verifiaction of feature engineering.
save_dataset( "NN_STFWI_df", NN_STFWI_df )
save_dataset( "NN_STM_df", NN_STM_df )
save_dataset( "NN_FWI_df", NN_FWI_df )
save_dataset( "NN_M_df", NN_M_df )

# Define All Hyperparameters
Define all the hyperparameters from the paper that will be used in the models.

There hyperparameters are declared in a single location to aid in marking.

In [27]:
# Multiple Regression
fit_intercept = True
copy_X = True
n_jobs = None
positive = False

# Create the Results DataFrame

In [28]:
results_df = pd.DataFrame(columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])

# Calculate the Naive results

In [29]:
# Generate an array with the predicted values being the mean of the area
y = area_df["area"]
y_pred = np.mean(y)
y_naive_pred = np.repeat(y_pred, len(y))

# Calculate the naive RMSE
naive_rmse = mean_squared_error(y, y_naive_pred, squared=False)
#print(naive_rmse)

# Add the naive results to the Results Table
rmse_df = pd.DataFrame([['Naive', naive_rmse, naive_rmse, naive_rmse, naive_rmse]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
results_df = results_df.append(rmse_df)

In [30]:
results_df.head()

,Model,STFWI (RMSE),STM (RMSE),FWI (RMSE),M (RMSE)
0,Naive,63.594226,63.594226,63.594226,63.594226


# Decision Tree

Scoring details:

RMSE: https://scikit-learn.org/stable/modules/model_evaluation.html

https://stackoverflow.com/questions/62514395/score-obtained-from-cross-val-score-is-rmse-or-mse


In [31]:
from sklearn.model_selection import train_test_split

print("DECISION TREE")
print("*"*30)

# Get the list of training / validating datasets
train_set = { "STFWI" : DT_STFWI_df, "STM" : DT_STM_df, "FWI" : DT_FWI_df, "M" : DT_M_df}

rmse_score = []

# Iterate over each dataset, fit the mode and evaluate the performance with RMSE
for key in train_set:
    print("="*10)
    print(key)

    # Run 10 train / test splits and take the average RMSE for each dataset
    results = []
    for run in range(10):
        # Split the dataset for train/test
        X_train, X_test, y_train, y_test = train_test_split( train_set[key], y_xformed, test_size=0.30, random_state=8)

        # Set hyperparameters according to the research paper
        # min_samples_split = 2
        model = DecisionTreeRegressor(min_samples_split = 2)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        rmse = calc_rmse( y_pred, y_test )
        results.append(rmse)

    avg = sum(results) / len(results)
    rmse_score.append(round(avg,1))
    print(results)
    print(f"RMSE: {avg}")

rmse_df = pd.DataFrame([['DT', rmse_score[0], rmse_score[1], rmse_score[2], rmse_score[3]]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
results_df = results_df.append(rmse_df)
    
del rmse_score, train_set

DECISION TREE
******************************
STFWI
[68.60289277625044, 68.61342740095128, 71.98437906183901, 67.86288984255339, 69.74762448691014, 68.5019831406481, 70.85038956220262, 68.89006143077812, 70.15138318262598, 69.78277815304358]
RMSE: 69.49878090378027
STM
[75.00149624047214, 75.74345708636095, 76.29902797686334, 74.08608447819036, 74.98926164395955, 77.39698205647593, 74.07188746428614, 75.77724543543991, 76.0565190673472, 76.15529256880191]
RMSE: 75.55772540181974
FWI
[65.82446232532854, 65.82808767311684, 65.82491631226056, 65.85872097716897, 65.85875156853201, 65.82682785087816, 65.8234203438463, 65.82457249957784, 65.82576295564097, 65.85876399407238]
RMSE: 65.83542865004225
M
[66.07441306771887, 65.12277339897639, 70.25640607560756, 69.44912358014338, 69.2303121844446, 70.34907963462084, 69.57117834064893, 70.50303885732019, 70.47973161757947, 66.18429302030732]
RMSE: 68.72203497773675


# Random Forrest

In [32]:
from sklearn.ensemble import RandomForestRegressor

print("RANDOM FORREST")
print("*"*30)


# Get the list of training / validating datasets
train_set = { "STFWI" : RF_STFWI_df, "STM" : RF_STM_df, "FWI" : RF_FWI_df, "M" : RF_M_df}

rmse_score = []

for key in train_set:
    print(key)
    print("="*10)
    
    # Run 10 train / test splits and take the average RMSE for each dataset
    results = []
    for run in range(10):
        # Split the dataset for train/test
        X_train, X_test, y_train, y_test = train_test_split( train_set[key], y_xformed, test_size=0.30, random_state=global_random_state)

        # Set hyperparameters according to research paper
        # Number of trees: 500
        model = RandomForestRegressor(n_estimators = 500)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        rmse = calc_rmse( y_pred, y_test )
        results.append(rmse)
    
    avg = sum(results) / len(results)
    rmse_score.append(round(avg,1))
    print(results)
    print(f"RMSE: {avg}")

    
rmse_df = pd.DataFrame([['RF', rmse_score[0], rmse_score[1], rmse_score[2], rmse_score[3]]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
results_df = results_df.append(rmse_df)

del rmse_score, train_set, rmse_df     # Delete the variables you do not want to be picked up accidentally elsewhere.

RANDOM FORREST
******************************
STFWI
[68.21576005758921, 68.30727058780327, 68.2809390991698, 68.12084892262068, 68.26885109610556, 68.2192997084707, 68.1816269307477, 68.36955895389741, 68.27402272637615, 68.38518445189084]
RMSE: 68.26233625346713
STM
[68.44030467776125, 68.35043607019931, 68.32971306425942, 68.41733628185452, 68.36953891092827, 68.34802109172702, 68.4311988804669, 68.36521772829029, 68.36505363846828, 68.3124074620013]
RMSE: 68.37292278059566
FWI
[68.09328354203925, 68.16132606266692, 68.06502761593111, 68.11827848877056, 68.14733927264393, 68.1133292180946, 68.06905993073687, 68.15525931373146, 68.12468853477799, 68.10965681722188]
RMSE: 68.11572487966146
M
[66.61726764340953, 66.29732552376684, 66.7814270377607, 66.39050011007649, 66.46796540495745, 66.651091097478, 66.63770696399763, 66.51573096360883, 66.59718972908007, 66.40655773479394]
RMSE: 66.53627622089296


# Multiple Regression

In [33]:
from sklearn.linear_model import LinearRegression

print("MULTIPLE REGRESSION")
print("*"*30)

# Get the list of training / validating datasets
train_set = { "STFWI" : MR_STFWI_df, "STM" : MR_STM_df, "FWI" : MR_FWI_df, "M" : MR_M_df}

rmse_score = []

for key in train_set:
    print(key)
    print("="*10)

    # Run 10 train / test splits and take the average RMSE for each dataset
    results = []
    for run in range(10):    
        # Split the dataset for train/test
        X_train, X_test, y_train, y_test = train_test_split( train_set[key], y_xformed, test_size=0.30, random_state=global_random_state)

        # Set hyperparameters according to research paper.
        # Use Least squares method.  Nothing to do. From scikit documentation: 
        # From the implementation point of view, this is just plain Ordinary Least Squares (scipy.linalg.lstsq)"
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        rmse = calc_rmse( y_pred, y_test )
        results.append(rmse)
    
    avg = sum(results) / len(results)
    rmse_score.append(round(avg,1))
    print(results)
    print(f"RMSE: {avg}")


rmse_df = pd.DataFrame([['MR', rmse_score[0], rmse_score[1], rmse_score[2], rmse_score[3]]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
results_df = results_df.append(rmse_df)

del rmse_score, train_set, rmse_df     # Delete the variables you do not want to be picked up accidentally elsewhere.

MULTIPLE REGRESSION
******************************
STFWI
[68.620656895443, 68.620656895443, 68.620656895443, 68.620656895443, 68.620656895443, 68.620656895443, 68.620656895443, 68.620656895443, 68.620656895443, 68.620656895443]
RMSE: 68.62065689544302
STM
[68.6124200522781, 68.6124200522781, 68.6124200522781, 68.6124200522781, 68.6124200522781, 68.6124200522781, 68.6124200522781, 68.6124200522781, 68.6124200522781, 68.6124200522781]
RMSE: 68.61242005227811
FWI
[68.67921183586752, 68.67921183586752, 68.67921183586752, 68.67921183586752, 68.67921183586752, 68.67921183586752, 68.67921183586752, 68.67921183586752, 68.67921183586752, 68.67921183586752]
RMSE: 68.6792118358675
M
[68.67760089392732, 68.67760089392732, 68.67760089392732, 68.67760089392732, 68.67760089392732, 68.67760089392732, 68.67760089392732, 68.67760089392732, 68.67760089392732, 68.67760089392732]
RMSE: 68.67760089392732


# SVM

In [34]:
from sklearn.svm import SVR

print("SVM")
print("*"*30)

# Get the list of training / validating datasets
train_set = { "STFWI" : SVM_STFWI_df, "STM" : SVM_STM_df, "FWI" : SVM_FWI_df, "M" : SVM_M_df}
# The hyperparameters to use for each dataset
gamma = { "STFWI" : 2.0e-5 , "STM" : 2.0e-3, "FWI" : 2.0e-3, "M" : 2.0e-2}


rmse_score = []

for key in train_set:
    print(key)
    print("="*10)

    # Run 10 train / test splits and take the average RMSE for each dataset
    results = []
    for run in range(10):        
        # Split the dataset for train/test
        X_train, X_test, y_train, y_test = train_test_split( train_set[key], y_xformed, test_size=0.30, random_state=global_random_state)

        # Set hyperparameters according to research paper.
        model = SVR(gamma=gamma[key])
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        rmse = calc_rmse( y_pred, y_test )
        results.append(rmse)
    
    avg = sum(results) / len(results)
    rmse_score.append(round(avg,1))
    print(results)
    print(f"RMSE: {avg}")

rmse_df = pd.DataFrame([['SVM', rmse_score[0], rmse_score[1], rmse_score[2], rmse_score[3]]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
results_df = results_df.append(rmse_df)

del rmse_score, train_set, rmse_df     # Delete the variables you do not want to be picked up accidentally elsewhere.

SVM
******************************
STFWI
[69.04385097543786, 69.04385097543786, 69.04385097543786, 69.04385097543786, 69.04385097543786, 69.04385097543786, 69.04385097543786, 69.04385097543786, 69.04385097543786, 69.04385097543786]
RMSE: 69.04385097543786
STM
[68.99591217033749, 68.99591217033749, 68.99591217033749, 68.99591217033749, 68.99591217033749, 68.99591217033749, 68.99591217033749, 68.99591217033749, 68.99591217033749, 68.99591217033749]
RMSE: 68.9959121703375
FWI
[68.98176471660854, 68.98176471660854, 68.98176471660854, 68.98176471660854, 68.98176471660854, 68.98176471660854, 68.98176471660854, 68.98176471660854, 68.98176471660854, 68.98176471660854]
RMSE: 68.98176471660852
M
[69.01166211185821, 69.01166211185821, 69.01166211185821, 69.01166211185821, 69.01166211185821, 69.01166211185821, 69.01166211185821, 69.01166211185821, 69.01166211185821, 69.01166211185821]
RMSE: 69.0116621118582


# Display Final Results

In [35]:
results_df.round(1)

,Model,STFWI (RMSE),STM (RMSE),FWI (RMSE),M (RMSE)
0,Naive,63.6,63.6,63.6,63.6
0,DT,69.5,75.6,65.8,68.7
0,RF,68.3,68.4,68.1,66.5
0,MR,68.6,68.6,68.7,68.7
0,SVM,69.0,69.0,69.0,69.0


# Question 2

Design and develop your own ML solution for this problem. The proposed solution should be different from all approaches mentioned in the provided article. This does not mean that you must have to choose a new ML algorithm. You can develop a novel solution by changing the feature selection approach or parameter optimisations process of used ML methods or using different ML methods or adding regularization or different combinations of them. This means, the proposed system should be substantially different from the methods presented in the article but not limited to only change of ML methods. Compare the RMSE result with reported methods in the article. Write in your report summarising your solution design and outcomes. The report should include:

1. Motivation behind the proposed solution.
2. How the proposed solution is different from existing ones.
3. Detail description of the model including all parameters so that any reader can implement your model.
4. Description of experimental protocol.
5. Evaluation metrics. 
6. Present results using tables and graphs. 
7. Compare and discuss results with respect to existing literatures.
8. Appropriate references (IEEE numbered).
